In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# Ruta del dataset
extracted_dir = r"C:\Users\alexa\OneDrive\Desktop\DataSetSignos"
classes = os.listdir(extracted_dir)
images = []
labels = []

In [ ]:
# Cargar imágenes y etiquetas
for i, sign_class in enumerate(classes):
    class_dir = os.path.join(extracted_dir, sign_class)
    if os.path.isdir(class_dir):
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            if os.path.isfile(image_path):
                image = cv2.imread(image_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                image = cv2.resize(image, (32, 32))
                images.append(image)
                labels.append(i)

images = np.array(images)
labels = np.array(labels)

In [ ]:
# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Expande las dimensiones de los datos de entrada
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

# Normalización de los datos
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
# Realizar aumento de datos
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
datagen.fit(X_train)

In [ ]:
# Definir el modelo
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(classes), activation='softmax'))

# Compilar el modelo
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])


In [ ]:
# Entrenar el modelo con aumento de datos
history = model.fit(datagen.flow(X_train, y_train, batch_size=32), 
                    steps_per_epoch=len(X_train) // 32, 
                    epochs=30, 
                    validation_data=(X_test, y_test))

# Evaluación del modelo en el conjunto de prueba
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Precisión en el conjunto de prueba:", test_accuracy)

In [ ]:
# Visualización de una imagen del dataset
pic = Image.open(r"C:\Users\alexa\OneDrive\Desktop\DataSetSignos\L\IMG_0966.JPG")
pic = pic.convert("L")
pic = pic.resize((32, 32))
pic_arr = np.array(pic) / 255.0
pic_arr = np.expand_dims(pic_arr, axis=0)
prediction = model.predict(pic_arr)
predicted_class = np.argmax(prediction)

print("Predicción:", predicted_class)

predicted_class_name = classes[predicted_class]
print("Clase predicha:", predicted_class_name)

In [ ]:
# Visualización de una imagen dentro del dataset
k = 2
pic = Image.open("C:\\Users\\alexa\\OneDrive\\Desktop\\DataSetSignos\\L\\IMG_0966.JPG")
pix = np.array(pic)
print(type(pic))

plt.imshow(pic.resize((32 * k, 32 * k)))
plt.show()

# Gráfico para la precisión
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Accuracy")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Test'])
plt.show()

# Gráfico para la pérdida
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Loss")
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Test'])
plt.show()
